In [49]:
import os
from dotenv import load_dotenv
load_dotenv()

# Groq + Langsmith Keys
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = os.getenv("LANGCHAIN_PROJECT")

In [50]:

## Data Ingestion
from langchain_community.document_loaders import WebBaseLoader
#loader = WebBaseLoader("https://docs.smith.langchain.com/tutorials/Administrators/manage_spend")
#loader = WebBaseLoader("https://python.langchain.com/docs/langsmith/guides/manage_spend/")
loader = WebBaseLoader("https://www.geeksforgeeks.org/artificial-intelligence/the-complete-langchain-ecosystem/")


docs = loader.load()

In [51]:
docs

[Document(metadata={'source': 'https://www.geeksforgeeks.org/artificial-intelligence/the-complete-langchain-ecosystem/', 'title': 'The Complete LangChain Ecosystem - GeeksforGeeks', 'description': 'Your All-in-One Learning Portal: GeeksforGeeks is a comprehensive educational platform that empowers learners across domains-spanning computer science and programming, school education, upskilling, commerce, software tools, competitive exams, and more.', 'language': 'en-US'}, page_content='\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nThe Complete LangChain Ecosystem - GeeksforGeeks\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSkip to content\n\n\n\n\n\n\n\n\n\n\n \n\n\n\n\n\n\n\n\n\n\nCoursesDSA / PlacementsGATE 2026 PrepML & Data ScienceDevelopmentCloud / DevOpsProgramming LanguagesAll CoursesTutorialsPythonJavaDSAML & Data ScienceInterview CornerProgramming LanguagesWeb DevelopmentGATECS SubjectsDevOpsScho

In [52]:
## Split documents
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
documents = text_splitter.split_documents(docs)

In [53]:
documents

[Document(metadata={'source': 'https://www.geeksforgeeks.org/artificial-intelligence/the-complete-langchain-ecosystem/', 'title': 'The Complete LangChain Ecosystem - GeeksforGeeks', 'description': 'Your All-in-One Learning Portal: GeeksforGeeks is a comprehensive educational platform that empowers learners across domains-spanning computer science and programming, school education, upskilling, commerce, software tools, competitive exams, and more.', 'language': 'en-US'}, page_content="The Complete LangChain Ecosystem - GeeksforGeeks\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSkip to content\n\n\n\n\n\n\n\n\n\n\n \n\n\n\n\n\n\n\n\n\n\nCoursesDSA / PlacementsGATE 2026 PrepML & Data ScienceDevelopmentCloud / DevOpsProgramming LanguagesAll CoursesTutorialsPythonJavaDSAML & Data ScienceInterview CornerProgramming LanguagesWeb DevelopmentGATECS SubjectsDevOpsSchool LearningSoftware and ToolsPracticePractice Coding ProblemsNation Skillup- Free CoursesProblem o

In [54]:
## Embeddings (HuggingFace instead of OpenAI)
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstoredb = FAISS.from_documents(documents, embeddings)

In [55]:
## Query Test
query = "What is LangChain Core and what roles does it serve?"
result = vectorstoredb.similarity_search(query)
print("Retrieved Context:\n", result[0].page_content)

Retrieved Context:
 LangChain is a framework that simplifies the process of building applications using Large Language Models (LLMs). It provides a set of modular building blocks that developers can use to design, integrate and deploy intelligent systems. The architecture of LangChain is structured in a way that separates the core logic, integration layers and advanced deployment tools. This layered approach makes LangChain both flexible for experimentation and robust for production use.LangChain ArchitectureCore Architecture of LangChainLet’s see how LangChain is structured from its core modules to advanced tools:1. LangChain Core (langchain-core)Provides the fundamental abstractions of the framework.Includes prompts, messages, models, chains, memory and output parsers.Does not depend on external providers, ensuring a lightweight base.Role: The foundation on which other layers are built.2. Main LangChain Package (langchain)Builds on langchain-core by offering higher-level workflows.Pr

In [56]:

## LLM (Groq)
from langchain_groq import ChatGroq
llm = ChatGroq(model="llama-3.1-8b-instant", temperature=0.7)

In [57]:
## Retrieval Chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    """
Answer the following question based only on the provided context:
<context>
{context}
</context>
"""
)



In [58]:

document_chain = create_stuff_documents_chain(llm, prompt)

In [59]:
from langchain.chains import create_retrieval_chain
retriever = vectorstoredb.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [60]:
## Final Response
response = retrieval_chain.invoke({"input": "What key components of LangChain?"})
print("\nAnswer:\n", response["answer"])


Answer:
 Based on the provided context, the LangChain ecosystem consists of the following components:

1. **LangChain Core (langchain-core)**: The foundational module that provides fundamental abstractions, including prompts, messages, models, chains, memory, and output parsers.

2. **Main LangChain Package (langchain)**: Builds on langchain-core by offering higher-level workflows, including orchestration for chains and agents, and remains model-agnostic.

3. **Integration Packages**: Independent connectors for external tools and services. Examples include langchain-openai and langchain-pinecone.

4. **LangChain Community (langchain-community)**: A collection of community-maintained integrations that expands LangChain's reach beyond official packages.

5. **LangGraph**: A framework for building stateful applications as computational graphs, supporting memory management, streaming outputs, and complex decision flows.

6. **LangServe**: Used for deploying LangChain applications as APIs,